In [114]:
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
import numpy as np
import import_ipynb
import utils_cdt as cdt
import pickle
import plotly.express as px
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

#613-F613 - Juros de Mora
*  SaldoAtualFinal
*  ValorMinimoExtrato
*  VencimentoBaseJurosReal
*  RotativoPagaJuros

In [115]:
infoDataBases = {"ip":"AZ2D-MASKDB-220.DEVCDT.COM.BR","base":"Controly"}

In [116]:
consulta = cdt.resultQuery("mdl_jurosmora_saldos.sql")
df_historico = cdt.extrairDados(infoDataBases,consulta,path='../data/',arquivo="mdl_jurosmora_"+infoDataBases["base"],spark=False,objSpark=None)

In [117]:
df_historico.head(2)

,percentual_cred,Id_Conta,SaldoExtratoAnterior,ValorMinimoExtrato,NumeroCiclo,FinanciavelExtrato,NaoFinanciavelExtrato,ParcelasFinanciaveis,ComprasNacionais,EncargosFinanciamento,...,ComprasSemJurosNaoVencidas,ParcelasComJurosNaoVencidas,SaldoFuturoJuros,SaldoFuturoParceladoComJuros,SaldoFuturoParceladoSemJuros,EncargosCiclo,TaxaDiariaCiclo,SaldoEntradaCreliq,SaldoJurosAcordo,FlagSMSExtratoIncondicional
0,0.0000,173875,457.62,68.64,22,2103.38,0.0,0.0,0.00,0.0,...,0.00,0.0,0.0,0.0,0.0,65.7295,0.004633,NaN,0.0,None
1,13.0636,173957,653.18,-555.20,21,16564.11,0.0,0.0,387.97,0.0,...,2631.98,0.0,0.0,0.0,0.0,110.2020,0.004633,NaN,0.0,None


In [118]:
df_historico.shape

(350962, 40)

In [119]:
consulta = cdt.resultQuery("mdl_jurosmora_transacoes.sql")
df_transacoes = cdt.extrairDados(infoDataBases,consulta,path='../data/',arquivo="mdl_jurosmora_tr_"+infoDataBases["base"],spark=False,objSpark=None)

In [120]:
df_transacoes.head(2)

,Id_Transacao,Id_Conta,Valor,DataVencimentoPadrao
0,119222793,518785,8.77,05/04/2020
1,119222794,518905,3.47,05/04/2020


In [121]:
# Faço um left join entre a tabela de historicos e transações, onde eu quero todo o faturamento independente se houve ou não lançamento de Juros Mora.
# Desta forma eu consigo prever quem teve lançamento ou não.

df = df_historico.merge(df_transacoes, on=['Id_Conta'], how='left')
df = df.fillna(0)

In [122]:
df.head(2)

,percentual_cred,Id_Conta,SaldoExtratoAnterior,ValorMinimoExtrato,NumeroCiclo,FinanciavelExtrato,NaoFinanciavelExtrato,ParcelasFinanciaveis,ComprasNacionais,EncargosFinanciamento,...,SaldoFuturoParceladoComJuros,SaldoFuturoParceladoSemJuros,EncargosCiclo,TaxaDiariaCiclo,SaldoEntradaCreliq,SaldoJurosAcordo,FlagSMSExtratoIncondicional,Id_Transacao,Valor,DataVencimentoPadrao
0,0.0000,173875,457.62,68.64,22,2103.38,0.0,0.0,0.00,0.0,...,0.0,0.0,65.7295,0.004633,0.0,0.0,0,0.0,0.0,0
1,13.0636,173957,653.18,-555.20,21,16564.11,0.0,0.0,387.97,0.0,...,0.0,0.0,110.2020,0.004633,0.0,0.0,0,0.0,0.0,0


In [123]:
# Remove colunas que contem ID
cols_id = [c for c in df.columns if c.lower()[:2] == 'id']
df = df.drop(columns=cols_id)


# Mantem apenas colunas numericas
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
df = df.select_dtypes(include=numerics)

In [124]:
df.head(2)

,percentual_cred,SaldoExtratoAnterior,ValorMinimoExtrato,NumeroCiclo,FinanciavelExtrato,NaoFinanciavelExtrato,ParcelasFinanciaveis,ComprasNacionais,EncargosFinanciamento,CreditosNacionais,...,ParcelasComJurosNaoVencidas,SaldoFuturoJuros,SaldoFuturoParceladoComJuros,SaldoFuturoParceladoSemJuros,EncargosCiclo,TaxaDiariaCiclo,SaldoEntradaCreliq,SaldoJurosAcordo,FlagSMSExtratoIncondicional,Valor
0,0.0000,457.62,68.64,22,2103.38,0.0,0.0,0.00,0.0,0.00,...,0.0,0.0,0.0,0.0,65.7295,0.004633,0.0,0.0,0,0.0
1,13.0636,653.18,-555.20,21,16564.11,0.0,0.0,387.97,0.0,653.18,...,0.0,0.0,0.0,0.0,110.2020,0.004633,0.0,0.0,0,0.0


In [125]:
preditoras =  df.columns.to_list()
preditoras.remove("Valor")
alvo = ['Valor']

In [126]:
X, Y = cdt.preperData(df,preditoras,alvo)

In [127]:
X_treino, Y_treino, X_teste, Y_teste = cdt.preperTreinoTeste(df, preditoras, alvo, 0.3)

In [128]:
# Treinando modelo
model = RandomForestRegressor(max_features='sqrt',n_estimators=100, max_depth=15).fit(X_treino, Y_treino.ravel())

In [129]:
model.score(X_teste,Y_teste)

0.9074641648083279

In [130]:
#Retirando metrcias do modelo 

Y_Predict = model.predict(X_teste)
Y_Predict = Y_Predict.round(2)
Y_Predict = Y_Predict.reshape(Y_Predict.shape[0],1)
df_final = cdt.transformDataSet(Y_Predict, Y_teste)
cdt.getresultdataset(df_final, "RandomForestRegressor")

SE - Sum of Error: 
[RandomForestRegressor] - Total Real:  46843.00
[RandomForestRegressor] - Total Prev:  46660.21
[RandomForestRegressor] - SE Final:  -182.79
[RandomForestRegressor] - Total Porcentagem de diferença do previsto x real:  0.39%
----------------------------------------------------------
ME - Mean Error:  
[RandomForestRegressor] - Total Registros: 51546
[RandomForestRegressor] - Total Diff : -183
[RandomForestRegressor] - ME Final : -0.003546
----------------------------------------------------------
MAE - Mean Absolute Error:  
[RandomForestRegressor] - Total Registros: 51546
[RandomForestRegressor] - Total Absolute Diff : 14877
[RandomForestRegressor] - ME Final : 0.288613
----------------------------------------------------------
MPE - Mean Percentage Error:  
[RandomForestRegressor] - Total Registros: 51546
[RandomForestRegressor] - Total percent Diff: 177310
[RandomForestRegressor] - MPE Final : 3.44%
----------------------------------------------------------
MAPAE

## Validando a eficacia do modelo com novos dados

In [140]:
consulta = cdt.resultQuery("estadoscontas_jurosmora.sql")
df_estadoscontas = cdt.extrairDados(infoDataBases,consulta,path='../data/',arquivo="jurosmora_"+infoDataBases["base"],spark=False,objSpark=None)

In [141]:
# Remove colunas que contem ID
cols_id = [c for c in df_estadoscontas.columns if c.lower()[:2] == 'id']
df_estadoscontas = df_estadoscontas.drop(columns=cols_id)
df_estadoscontas.round(2)

df_estadoscontas = df_estadoscontas.replace([np.inf, -np.inf], np.nan)
df_estadoscontas = df_estadoscontas.fillna(0)

In [142]:
consulta = cdt.resultQuery("transacoescorrentes_jurosmora.sql")
df_lancamentos = cdt.extrairDados(infoDataBases,consulta,path='../data/',arquivo="jurosmora_tr_"+infoDataBases["base"],spark=False,objSpark=None)

In [143]:
df_lancamentos.shape

(5593, 4)

In [144]:
df_lancamentos.round(2)
Y_real = df_lancamentos['Valor'].values

In [145]:
X = np.absolute(df_estadoscontas[preditoras].values)

In [146]:
Y_Predict = model.predict(X)

In [152]:
print(f" Total previsto: R${Y_Predict.sum()}")
print(f" Total real: R${Y_real.sum()} ")
print(f" Diferença total: R${Y_real.sum()-Y_Predict.sum()} ")
print(" Porcentagem diferença: {:.2f}%".format((((Y_real.sum()-Y_Predict.sum()) / Y_real.sum())*100)))

 Total previsto: R$15215.45487980714
 Total real: R$15925.21 
 Diferença total: R$709.75512019286 
 Porcentagem diferença: 4.46%
